# Oil prediction analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import xgboost as xgb
import seaborn as sns
import operator as op
import typing as t
import statistics
import shap

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.tree import DecisionTreeRegressor

from sklearn.feature_selection import RFE
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn import decomposition
from sklearn.decomposition import PCA

from sklearn.datasets import make_classification

from sklearn.neural_network import MLPRegressor

from sklearn.svm import SVR

from sklearn.preprocessing import MinMaxScaler

from __future__ import annotations

from collections import abc

from functools import partial

from tqdm.auto import trange
from tqdm import trange

from xgboost import XGBClassifier
from xgboost import XGBRegressor

from eBoruta import eBoruta, TrialData, Features, Dataset, setup_logger  
# pip install git+https://github.com/edikedik/eBoruta.git

ModuleNotFoundError: No module named 'shap'

# Load and analysis of the data

We selected some parameters that could influence the price of oil such as:  
'Year', 'Price Cononut oil', 'Price Sugar', 'Us crude oil reserves', 'Quantity oil embarked', 'Quantity goods embarked', 'Pandemic (covid)', 'War','Electric car registrations', 'World imports', 'World exports', 'Value of Solar Consumption', 'Inflation', 'Value of Wind Consumption', 'Value of Nuclear Consumption', 'Value of Natural Gas Consumption', 'Population', 'OPEC cuts on production', 'Price gold', 'GDP Growth','Crude oil and NGL production', 'World-oil demand', 'Value of Freight Transport'.

In [ ]:
# Load the csv
df = pd.read_csv('data_final1.csv')

x = df.drop(['Oil brent price ($/bbl)'], 1)
y = df['Oil brent price ($/bbl)']

# Normalize x
min_vals = x.min()
max_vals = x.max()
x = (x - min_vals) / (max_vals - min_vals)

display(df.describe())

corr_matrix = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

df.hist(figsize=(10, 8), bins=20)
plt.tight_layout()
plt.show()


plt.figure(figsize=(10, 6))
plt.plot(df['Year'], df['Oil brent price ($/bbl)'])
plt.xlabel('Date')
plt.ylabel('Oil brent price ($/bbl)')
plt.title('Oil brent price ($/bbl) over time')
plt.xticks(rotation=45)
plt.show()


# Explore different models 

In [ ]:
def cross_validate(model, x, y, n, score_fn, agg_fn, stratified):
    if stratified:
        kf = StratifiedKFold(n_splits=n)
    else:
        kf = KFold(n_splits=n)

    scores = []
    for train_idx, test_idx in kf.split(x, y):
        X_train, Y_train = x.iloc[train_idx], y.iloc[train_idx]
        X_test, Y_test = x.iloc[test_idx], y.iloc[test_idx]
        model.fit(X_train, Y_train)
        y_pred = model.predict(X_test)
        score = score_fn(Y_test, y_pred)
        scores.append(score)
    return agg_fn(scores)

    

In [ ]:
print(x.columns)
df = pd.read_csv('data_final1.csv')
x = df.loc[:, df.columns != 'Oil brent price ($/bbl)']
y = df['Oil brent price ($/bbl)']
selected_columns = ['Year', 'World imports', 'Price gold', 'War', 'Inflation']
# selected_columns2 = ['Year', 'World imports', 'Cononut oil ($/mt)', 'Sugar ($/kg)', 'Inflation']
xs = x.loc[:, selected_columns]

# define models to cross-validate
models = []
models.append(('XGBRegressor',XGBRegressor(),mean_squared_error,np.mean, False))
#models.append(('DecisionTree', DecisionTreeRegressor(),mean_squared_error,np.mean, False)) 
models.append(('RandomForest', RandomForestRegressor(),mean_squared_error,np.mean, False))
#models.append(('LogisticRegression', LogisticRegression(),mean_squared_error,np.mean, False))
stratified=0

# evaluate each model using cross-validation
results = []
n=4
for name, model, score_fn, agg_fn, stratified in models:
    print (name, model, score_fn, agg_fn, stratified )
    score=cross_validate(model, x, y, n, score_fn, agg_fn, stratified)
    score1=cross_validate(model, xs, y, n, score_fn, agg_fn, stratified)
    # results.append(score)
    print(f'x: {score}')
    print(f'xs: {score1}')

# best_model = models[np.argmin(results)]
# print(f'Best_model={best_model}')


## PCA transformation on data

In [ ]:
#Import and normalize the data

df = pd.read_csv('data_final1.csv')
x = df.loc[:, df.columns != 'Oil brent price ($/bbl)']
y = df['Oil brent price ($/bbl)']

x = (x - x.min()) / (x.max() - x.min())

In [ ]:
pca = PCA()
x_pca = pca.fit_transform(x)

explained_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
print(explained_variance_ratio)
print(len(explained_variance_ratio))

# Determine the number of components that explain 90% of the variance
n_components = np.argmax(explained_variance_ratio >= 0.9) + 1

pca = PCA(n_components=n_components)

x_pca = pca.fit_transform(x)

x_pca = pd.DataFrame(x_pca)

selected_features = pca.components_
print(len(selected_features))

We need 5 features to get 90% of the variance explained, just as many features as our selection.

In [ ]:
plt.plot(range(1, 10), explained_variance_ratio[:9])
plt.xlabel('Principal component')
plt.ylabel('Ratio of explained variance')
plt.axhline(y=0.9, color='red', linestyle='--')
plt.axvline(x=5, color='yellow', linestyle='--')
plt.axvline(x=4, color='green', linestyle='--')
plt.title('Ratio of explained variance over number of components')
plt.show()

In [ ]:
pca = decomposition.PCA(n_components=8)
pca.fit(x)
X_transformed = pca.transform(x)

plt.plot(range(1, 9), pca.explained_variance_ratio_)
plt.xlabel('Principal component')
plt.ylabel('Ratio of explained variance by new component')
plt.axhline(y=0.025, color='red', linestyle='--')
plt.axvline(x=5, color='yellow', linestyle='--')
plt.axvline(x=6, color='green', linestyle='--')
plt.title('Ratio of explained variance by new component over the number of components')
plt.show()

After 5 components, less than 2.5% of the variance is explained by adding a new parameter, another reason to keep 5 components.

In [ ]:
# define models to cross-validate
models = []
models.append(('XGBRegressor',XGBRegressor(),mean_squared_error,np.mean, False))
models.append(('DecisionTree', DecisionTreeRegressor(),mean_squared_error,np.mean, False)) 
models.append(('RandomForest', RandomForestRegressor(),mean_squared_error,np.mean, False))

models_PCA = []
models_PCA.append(('XGBRegressor',XGBRegressor(),mean_squared_error,np.mean, False))
models_PCA.append(('DecisionTree', DecisionTreeRegressor(),mean_squared_error,np.mean, False)) 
models_PCA.append(('RandomForest', RandomForestRegressor(),mean_squared_error,np.mean, False))
#models.append(('LogisticRegression', LogisticRegression(),mean_squared_error,np.mean, False))
stratified=0

# evaluate each model using cross-validation
results = []
n=4
for name, model, score_fn, agg_fn, stratified in models:
    print('raw data')
    print (name, model, score_fn, agg_fn, stratified )
    score=cross_validate(model, x, y, n, score_fn, agg_fn, stratified)
    results.append(score)
    print(score)
    
results_PCA = []
n=4
for name, model, score_fn, agg_fn, stratified in models_PCA:
    print('data transformed by PCA')
    print (name, model, score_fn, agg_fn, stratified )
    score=cross_validate(model, x_pca, y, n, score_fn, agg_fn, stratified)
    results_PCA.append(score)
    print(score)

best_model = models[np.argmin(results)]
print(f'Best_model={best_model}')
best_model_PCA = models_PCA[np.argmin(results_PCA)]
print(f'Best_model={best_model_PCA}')

First, notice that the model with the lowest mean squared error is XGB regressor in both cases.

We also see that using PCA with 5 components on our data is very efficient as it reduces our error from 621 to 253, more than divides it by 2.

# Feature selection

## Wrapper method  
##   -  Sequential Feature Selector "forward"

In [ ]:
estimators = [RandomForestRegressor, XGBRegressor]

for estimator in estimators: 
    selector = SequentialFeatureSelector(estimator=estimator(),
                                         n_features_to_select=5,
                                         direction='forward')

    selector.fit(x, y)


    selected_features = x.columns[selector.support_].tolist()
    

    print(selected_features)
    
    estimator = estimator()
    estimator.fit(x[selected_features], y)
    
    explainer = shap.TreeExplainer(estimator)
    shap_values = explainer.shap_values(x[selected_features])
    
    shap.summary_plot(shap_values, x[selected_features], feature_names=selected_features)
    
    selected_feature_importances = pd.DataFrame({'Feature': selected_features, 'Importance': np.abs(shap_values).mean(axis=0)})
    selected_feature_importances = selected_feature_importances.sort_values(by='Importance', ascending=False)
    top_5_features = selected_feature_importances['Feature'].head(5).tolist()
        
        
    print(f"Estimator: {estimator}")
    print(f"selected_features: {top_5_features}")
    
    print()





    

## -  Recursive Feature Elimination 

In [ ]:
estimators = [RandomForestRegressor, XGBRegressor]

for estimator in estimators: 
    model = estimator()
    rfe = RFE(estimator=model, n_features_to_select=5)
    rfe.fit(x, y)
    feature_indices = rfe.get_support(indices=True)
    selected_features = x.columns[feature_indices]

    selected_x = x[selected_features]
    model.fit(selected_x, y)
    
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(selected_x)
    shap.summary_plot(shap_values, selected_x, feature_names=selected_features)

    selected_feature_importances = pd.DataFrame({'Feature': selected_features, 'Importance': np.abs(shap_values).mean(axis=0)})
    selected_feature_importances = selected_feature_importances.sort_values(by='Importance', ascending=False)
    top_5_features = selected_feature_importances['Feature'].head(5).tolist()
        
        
    print(f"Estimator: {estimator}")
    print(f"selected_features: {top_5_features}")
    
    print()



## - Boruta

In [ ]:
def plot_imp_history(df_history: pd.DataFrame):
    sns.lineplot(x='Step', y='Importance', hue='Feature', data=df_history)
    sns.lineplot(x='Step', y='Threshold', data=df_history, linestyle='--', linewidth=4)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()
    
boruta = eBoruta(n_iter = 20, verbose=2, classification = False, test_stratify = False, pvalue = 0.001, percentile = 100) 
boruta.fit(x, y, model = XGBRegressor());
features = boruta.features_
print(f'features accepted: {features.accepted}')
print(f'features rejected: {features.rejected}')
print(f'features tentative: {features.tentative}')

df = features.history
plot_imp_history(df)
r=boruta.rank(sort = True)
print(r)
dictionary = dict(zip(r['Feature'], r['Importance'].apply(lambda x: [x])))
print(dictionary)


## Embelled method
## - XGboost

In [ ]:

boost = XGBRegressor()
boost.fit(x, y)
p_boost = boost.predict(x)

feature_importance = boost.feature_importances_

top_5_indices = feature_importance.argsort()[-5:][::-1]
top_5_features = x.columns[top_5_indices]
print(top_5_features)

x_top_5 = x[top_5_features]

boost.fit(x_top_5, y)

explainer = shap.TreeExplainer(boost)

shap_values = explainer.shap_values(x_top_5)

shap.summary_plot(shap_values, x_top_5, feature_names=x.columns)






## Filter method
## - Correlation 

In [ ]:
df = pd.read_csv('data_final1.csv')
correlation_matrix = df.corr()

threshold = 0.70

target_correlation = correlation_matrix['Oil brent price ($/bbl)']

target_correlation_sorted = target_correlation.sort_values(ascending=False)

selected_features = 0


print(target_correlation_sorted[1:6])

corr_with_target = df.corr()['Oil brent price ($/bbl)'].abs().sort_values(ascending=True)
d = corr_with_target.to_dict()
del d['Oil brent price ($/bbl)']
data = pd.Series(d)
plt.barh(data.index, data.values)
plt.xlabel('Correlation Coefficient')
plt.title(f'Correlation with Oil brent price ($/bbl)')
plt.show()




## - ANOVA F-v

In [ ]:

f_scores, p_values = f_classif(x, y)

feature_scores = pd.DataFrame({'Feature': x.columns, 'F-Score': f_scores, 'p-value': p_values})

feature_scores = feature_scores.sort_values('F-Score', ascending=False)

print(feature_scores[:5])


## Selection

According to the previous studies, the most important parameters, sort in descending order of importance, are:  
- Year
- World imports
- World exports
- Inflation
- Price gold
- War
- OPEC cuts on production

## Correlation

We want to avoid having 2 highly correlated features in our small selection. 

In [ ]:

corr_matrix = df.corr()
sns.heatmap(corr_matrix, cmap='coolwarm', annot=False)

plt.show()


In [ ]:
df2=df.copy()
df2.drop(['Oil brent price ($/bbl)'], 1)
corr_threshold = 0.95

corr_matrix = df.corr().abs()

highly_corr_features = set()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if corr_matrix.iloc[i, j] >= corr_threshold:
            feature_i = corr_matrix.columns[i]
            feature_j = corr_matrix.columns[j]
            highly_corr_features.add((feature_i, feature_j))

print("highly_corr_features: ", highly_corr_features)



Since "World exports" and "World imports" are highly correlated, we keep only one of the two (the more important one).  
Here is the final selection (the most important parameters):  
- Year
- World imports
- Inflation
- Price gold
- War

## Retraining with the selected features

Now we want to compare the PCA components with our selected features

In [ ]:
x_selected = x[['Year', 'World imports', 'Price gold', 'War', 'Inflation']]

x_selected

In [ ]:
# define models to cross-validate
models_PCA = []
models_PCA.append(('XGBRegressor',XGBRegressor(),mean_squared_error,np.mean, False))
models_PCA.append(('DecisionTree', DecisionTreeRegressor(),mean_squared_error,np.mean, False)) 
models_PCA.append(('RandomForest', RandomForestRegressor(),mean_squared_error,np.mean, False))

models_selected = []
models_selected.append(('XGBRegressor',XGBRegressor(),mean_squared_error,np.mean, False))
models_selected.append(('DecisionTree', DecisionTreeRegressor(),mean_squared_error,np.mean, False)) 
models_selected.append(('RandomForest', RandomForestRegressor(),mean_squared_error,np.mean, False))

# evaluate each model using cross-validation
    
results_PCA = []
n=4
for name, model, score_fn, agg_fn, stratified in models_PCA:
    print('Data transformed by PCA')
    print (name, model, score_fn, agg_fn, stratified )
    score=cross_validate(model, x_pca, y, n, score_fn, agg_fn, stratified)
    results_PCA.append(score)
    print(score)
    
results_selected = []
n=4
for name, model, score_fn, agg_fn, stratified in models_selected:
    print('Data with selected features')
    print (name, model, score_fn, agg_fn, stratified )
    score=cross_validate(model, x_selected, y, n, score_fn, agg_fn, stratified)
    results_selected.append(score)
    print(score)
    

best_model_PCA = models_PCA[np.argmin(results_PCA)]
print(f'Best_model={best_model_PCA}')
best_model_selected = models[np.argmin(results_selected)]
print(f'Best_model={best_model_selected}')

Once again, XGB regressor is the best model.

We notice that PCA has a better score than our selection: an error of 253 instead of an error of 372.

We also notice that using PCA on our selection is almost useless as we obtain a very similar error of 358.

If we only look at the scores, we deduce that PCA works better than our selection.

But there is a major flaw in this reasoning: PCA does not permit interpretability.
Indeed, the components selected by PCA do not represent any feature, they are created by PCA.

We conclude then that our selection of feature seems very good as it has a rather small error of 372 versus the error of the original data of 621, and it allows interpretability.

We have also used this to confirm that our selection of 5 parameters is very good as it has a lower error than other selections we have tested.

## Going further - Creating a Neural Network

In [ ]:
selected_columns = ['Year', 'World imports', 'Price gold', 'War', 'Inflation']
# selected_columns2 = ['Year', 'World imports', 'Cononut oil ($/mt)', 'Sugar ($/kg)', 'Inflation']
xs = x.loc[:, selected_columns]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(23,)))
model.add(tf.keras.layers.Dropout(0.2))  # Dropout regularization with a rate of 0.2

model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))  # Dropout regularization with a rate of 0.2

model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1))

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

loss = model.evaluate(X_test, y_test)
print(loss)

This new model we created has a better score for our selected data than XGB regressor (274 instead of 372).

However, we should keep in mind that neural networks are likely to overfit if we complexify them too much, which is why we only put 3 activation layers in this case.